# Matrix Decomposition

## Contents:
 * [Gaussian Decomposition](#Gaussian-Decomposition)
     * [$LU$]()
     * [$LDU$]()
     * [$LDL^t$]()
     * [$RR^t$]()
 * [Orthogonal Decomposition](#Orthogonal-Decomposition)
     * [$QR$ - Givens](#$A-=-QR$-decomposition-of-Householder)
     * [$QR$ - Householder](#$A-=-QR$-decomposition-of-Householder)
 * [Diagonal Decomposition](#Diagonal-Decomposition)
     * [$U \Sigma V^t$]()

#### Author:
* Sergio García Prado - [garciparedes.me](https://garciparedes.me)

In [4]:
from typing import Tuple
import enforce

In [3]:
import numpy as np

In [713]:
def gauss(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    Ab = np.column_stack((A, b))
    for i in range(Ab.shape[0]):
        if (Ab[i, i] != 0):
            for j in reversed(range(i + 1, Ab.shape[0])):
                Ab[j,:] -= Ab[j, i] / Ab[i, i] * Ab[i,:]
    print(Ab)
    
    for i in reversed(range(Ab.shape[0])):
        if (Ab[i, i] == 0):
            Ab[:i, i] = 0
        else:
            Ab[i, :] = Ab[i, :] / Ab[i, i] #- A[i, i + 1] / A[i + 1, i + 1] #* A[i+1,:]
        print(i)
            
            
    print(Ab)
    return Ab[:, -1]

In [714]:
A = np.array([[6,  3,  3],
              [3,  6, -3],
              [3, -3,  6]])
A = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])

b = np.ones(3)
gauss(A, b)

[[ 1.  2.  3.  1.]
 [ 0. -3. -6. -3.]
 [ 0.  0.  0.  0.]]
2
1
0
[[ 1.  2.  0.  1.]
 [-0.  1. -0.  1.]
 [ 0.  0.  0.  0.]]


array([1., 1., 0.])

## Gaussian Decomposition

#### $A = LU$ decomposition

In [553]:
@enforce.runtime_validation
def lu(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    A = A.astype(np.float64)
    for i in range(A.shape[0]):
        if (A[i, i] != 0):
            for j in reversed(range(i + 1, A.shape[0])):
                A[j, (i+1):] = A[j, (i+1):] - A[j, i] / A[i, i] * A[i, (i+1):]
                A[j, i] = A[j, i] / A[i, i]
    return (np.tril(A, -1) + np.eye(A.shape[0]), np.triu(A))

In [561]:
@enforce.runtime_validation
def gauss_lu(LU: Tuple[np.ndarray, np.ndarray], b: np.ndarray) -> np.ndarray:
    for i in range(LU[0].shape[0]):
        b[i] -= np.sum(b[:i] * LU[0][i,:i])
    for i in reversed(range(LU[1].shape[0])):
        if LU[1][i][i] != 0:
            b[i] = b[i] / LU[1][i][i] - np.sum(b[(i + 1):] * LU[1][i, (i + 1):])
    return b

In [598]:
A = np.array([[1, 2, 3],[4, 5, 6],[7, 8, 9]])

print(lu(A)[0] @ lu(A)[1])
gauss_lu(lu(A), np.ones(3))

[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


array([-1.,  1.,  0.])

In [307]:
L = np.array([[1, 0, 0, 0], 
              [1, 1, 0, 0],
              [2,-1, 1, 0],
              [-1,1, 0, 1]])

U = np.array([[1, 1, -1, 1], 
              [0, 1, -3, -3],
              [0, 0, 0, 0],
              [0, 0, 0, 0]])

P = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])
A = P @ L @ U
print(P @ L @ U)

[[ 1  1 -1  1]
 [ 1  2 -4 -2]
 [ 2  1  1  5]
 [-1  0 -2 -4]]


In [3]:
L = np.array([[  1,    0,   0,  0], 
              [ -1,    1,   0,  0],
              [3/2, 5/14,   1,  0],
              [  3,    0,   0,  1]])

U = np.array([[ 2,     3,   1,   2], 
              [ 0,     7,   0,   7],
              [ 0,     0,   0,-9/2],
              [ 0,     0,   0,   1]])

P = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

print(P @ L @ U)

[[ 2.   3.   1.   2. ]
 [-2.   4.  -1.   5. ]
 [ 3.   7.   1.5  1. ]
 [ 6.   9.   3.   7. ]]


In [4]:
L = np.array([[  1,   0,   0], 
              [3/4,   1,   0],
              [-1/2,  0,   1]])

U = np.array([[ 4,    2,   6], 
              [ 0, -3/2, 5/2],
              [ 0,     0, 0]])

P = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

print(P @ L @ U)

[[ 4.  2.  6.]
 [ 3.  0.  7.]
 [-2. -1. -3.]]


In [5]:
L = np.array([[  1,    0,   0,  0], 
              [1/2,   1,   0,  0],
              [  1,   -1,   1,  0],
              [3/2,   -1, 4/3,  1]])

U = np.array([[ 2,  0,   1,   2], 
              [ 0,  1,-1/2,   1],
              [ 0,  0, 3/2,   0],
              [ 0,  0,   0,   1]])

P = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

print(P @ L @ U)

[[ 2.  0.  1.  2.]
 [ 1.  1.  0.  2.]
 [ 2. -1.  3.  1.]
 [ 3. -1.  4.  3.]]


In [6]:
L = np.array([[1, 0, 0, 0], 
              [-1, 1, 0, 0],
              [3, 1/2, 1, 0],
              [2, 0, 0, 1]])

U = np.array([[1, 2, 0, -2], 
              [0, 2, 4, 5],
              [0, 0, 0, 1/2],
              [0, 0, 0, 1]])

P = np.array([[1, 0, 0, 0],
              [0, 0, 0, 1],
              [0, 0, 1, 0],
              [0, 1, 0, 0]])

print(P @ L @ U)

[[ 1.  2.  0. -2.]
 [ 2.  4.  0. -3.]
 [ 3.  7.  2. -3.]
 [-1.  0.  4.  7.]]


## Gaussian Decomposition

#### $A = LDU$ decomposition

In [29]:
@enforce.runtime_validation
def ldu(A: np.ndarray) -> Tuple[np.array, np.array, np.array]:
    L = np.zeros(A.shape)
    D = np.zeros(A.shape)
    U = np.zeros(A.shape)
    
    raise NotImplementedError("TODO IMPLEMENT YET: LDU")

In [7]:
L = np.array([[  1,   0,   0], 
              [  5,   1,   0],
              [ -2,   3,   1]])

U = np.array([[  4,  -2,   1], 
               [ 0,   3,   7],
               [ 0,   0,  -2]])

D = np.array([[ 4, 0, 0], 
              [0, 3, 0],
              [0, 0,-2]])

P = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

print(P @ L @ U)

[[ 4 -2  1]
 [20 -7 12]
 [-8 13 17]]


In [8]:
L = np.array([[1, 0, 0, 0], 
              [4, 1, 0, 0],
              [6, 8, 1, 0],
              [5, -5, 0, 1]])

U = np.array([[1, 2, -2, 1],
              [0, -3, 1, 2],
              [0, 0, -2, 0],
              [0, 0, 0, 8]])

D = np.array([[1, 0, 0, 0],
              [0, -3, 0, 0],
              [0, 0, -2, 0],
              [0, 0, 0, 8]])

P = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

print(P @ L @ U)

[[  1   2  -2   1]
 [  4   5  -7   6]
 [  6 -12  -6  22]
 [  5  25 -15   3]]


In [9]:
L = np.array([[  1,   0,   0], 
              [  5,   1,   0],
              [ -2,   3,   1]])

D = np.array([[ 4, 0, 0], 
              [0, 3, 0],
              [0, 0,-2]])

U = np.array([[ 1,-1/2,1/4], 
               [ 0,   1,7/3],
               [ 0,   0,  1]])


P = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

print(P @ L @ D @ U)

[[ 4. -2.  1.]
 [20. -7. 12.]
 [-8. 13. 17.]]


In [10]:
L = np.array([[1, 0, 0, 0], 
              [4, 1, 0, 0],
              [6, 8, 1, 0],
              [5, -5, 0, 1]])

D = np.array([[1, 0, 0, 0],
              [0, -3, 0, 0],
              [0, 0, -2, 0],
              [0, 0, 0, 8]])

U = np.array([[1, 2, -2, 1],
               [0, 1, -1/3, -2/3],
               [0, 0, 1, 0],
               [0, 0, 0, 1]])


P = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

print(P @ L @ D @ U)

[[  1.   2.  -2.   1.]
 [  4.   5.  -7.   6.]
 [  6. -12.  -6.  22.]
 [  5.  25. -15.   3.]]


In [11]:
L = np.array([[1, 0, 0, 0, 0], 
              [0, 1, 0, 0, 0],
              [4, 2, 1, 0, 0],
              [5, 3, -1, 1, 0],
              [0, 0, 0, 0, 1]])

U = np.array([[1, 2, -2, 1, 1],
              [0, 1, -1/3, -2/3, -2/3],
              [0, 0, 1, 0, -1/2],
              [0, 0, 0, 1, 1/2],
              [0, 0, 0, 0, 1]])

D = np.array([[1, 0, 0, 0, 0],
              [0, -3, 0, 0, 0],
              [0, 0, -2, 0, 0],
              [0, 0, 0, 2, 0],
              [0, 0, 0, 0, 2]])

P = np.array([[1, 0, 0, 0, 0],
              [0, 1, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1]])

print(P @ L @ D @ U)

[[ 1.  2. -2.  1.  1.]
 [ 0. -3.  1.  2.  2.]
 [ 4.  2. -8.  8.  9.]
 [ 5.  1. -5. 13. 11.]
 [ 0.  0.  0.  0.  2.]]


#### $A = LDL^t$ decomposition

In [28]:
@enforce.runtime_validation
def lu(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    L = np.zeros(A.shape)
    D = np.zeros(A.shape)
    
    raise NotImplementedError("TODO IMPLEMENT YET LDL")

In [12]:
L = np.array([[1, 0, 0], 
              [-1/2, 1, 0],
              [-2, -9/8, 1]])

D = np.array([[4, 0, 0], 
              [0, -8, 0],
              [0, 0, 89/8]])

P = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

print(P @ L @ D @ np.transpose(L))

[[ 4. -2. -8.]
 [-2. -7. 13.]
 [-8. 13. 17.]]


In [13]:
L = np.array([[1, 0, 0, 0], 
              [4, 1, 0, 0],
              [-2, 4/11, 1, 0],
              [1, -21/11, 227/94, 1]])

D = np.array([[1, 0, 0, 0], 
              [0, -11, 0, 0],
              [0, 0, -94/11, 0],
              [0, 0, 0, 8077/94]])

P = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

print(P @ L @ D @ np.transpose(L))

[[  1.   4.  -2.   1.]
 [  4.   5. -12.  25.]
 [ -2. -12.  -6. -15.]
 [  1.  25. -15.  -3.]]


In [14]:
L = np.array([[1, 0, 0, 0], 
              [4, 1, 0, 0],
              [-2, 4/11, 1, 0],
              [1, -21/11, 227/94, 1]])

D = np.array([[1, 0, 0, 0], 
              [0, -11, 0, 0],
              [0, 0, -94/11, 0],
              [0, 0, 0, 8077/94]])

P = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

print(P @ L @ D @ np.transpose(L))

[[  1.   4.  -2.   1.]
 [  4.   5. -12.  25.]
 [ -2. -12.  -6. -15.]
 [  1.  25. -15.  -3.]]


In [15]:
L = np.array([[1, 0, 0, 0, 0], 
              [2, 1, 0, 0, 0],
              [4, 6/7, 1, 0, 0],
              [5, 8/7, 127/132, 1, 0],
              [1, 0, -35/132, -1163/739, 1]])

D = np.array([[1, 0, 0, 0, 0], 
              [0, -7, 0, 0, 0],
              [0, 0, -132/7, 0, 0],
              [0, 0, 0, 739/132, 0],
              [0, 0, 0, 0,  -8528/739]])

P = np.array([[1, 0, 0, 0, 0],
              [0, 1, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1]])

print(P @ L @ D @ np.transpose(L))

[[ 1.  2.  4.  5.  1.]
 [ 2. -3.  2.  2.  2.]
 [ 4.  2. -8. -5.  9.]
 [ 5.  2. -5.  4.  1.]
 [ 1.  2.  9.  1.  2.]]


## $A = RR^t$ decomposition

In [27]:
@enforce.runtime_validation
def lu(A: np.ndarray) -> Tuple[np.ndarray]:
    R = np.zeros(A.shape)
    
    raise NotImplementedError("TODO IMPLEMENT YET: RR")

In [16]:
R = np.array([[2, 0, 0],
              [1, 3, 0],
              [-2, 2, 1]])

P = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

print(P @ R @ np.transpose(R))

[[ 4  2 -4]
 [ 2 10  4]
 [-4  4  9]]


In [17]:
R = np.array([[2, 0, 0],
              [1, np.sqrt(8), 0],
              [0, np.sqrt(8) / 2, np.sqrt(3)]])

P = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

print(P @ R @ np.transpose(R))

[[4. 2. 0.]
 [2. 9. 4.]
 [0. 4. 5.]]


## Orthogonal Decomposition

#### $A = QR$ decomposition of Gram Schmidt

#### $A = QR$ decomposition of Givens 

In [5]:
@enforce.runtime_validation
def rotator(A: np.ndarray, i: int, j: int, k: int) -> np.ndarray:
    G = np.eye(A.shape[0])
    r = np.sqrt(A[i, k] ** 2 + A[j, k] ** 2)
    if A[j, k] != 0:
        G[i, i] = A[i, k] / r
        G[i, j] = A[j, k] / r
        G[j, i] = - A[j, k] / r
        G[j, j] = A[i, k] / r
    return G

In [6]:
@enforce.runtime_validation
def qr_givens(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    Q = np.eye(A.shape[0])
    for j in range(A.shape[1]):
        G = np.eye(A.shape[0])
        for i in reversed(range(j, A.shape[1])):
            G = rotator(G @ A, i, i + 1, j) @ G
        Q = Q @ np.transpose(G)
        A = G @ A
    return (Q, A)

In [7]:
A = np.array([[1,-1,1],
              [1,1,1],
              [1,1,-1],
             [1,1,1]])

b = np.array([1, 1, -1, 0])
[Q, R] = qr_givens(A)
c = (np.transpose(Q) @ b).flatten()
x = np.linalg.solve(R[0:R.shape[1], :], c[0:R.shape[1]])

print("A =", np.round(A, decimals=2),  '\n', sep='\n')
print("Q =", np.round(Q, decimals=2),  '\n', sep='\n')
print("R =", np.round(R, decimals=2),  '\n', sep='\n')
print("Q @ R =", np.round(Q @ R, decimals=2), '\n', sep='\n')
print("b =", np.round(b, decimals=2), '\n', sep='\n')
print("c =", np.round(c, decimals=2), '\n', sep='\n')
print("x =", np.round(x, decimals=2), '\n', sep='\n')

A =
[[ 1 -1  1]
 [ 1  1  1]
 [ 1  1 -1]
 [ 1  1  1]]


Q =
[[ 0.5  -0.87  0.    0.  ]
 [ 0.5   0.29  0.41  0.71]
 [ 0.5   0.29 -0.82  0.  ]
 [ 0.5   0.29  0.41 -0.71]]


R =
[[ 2.    1.    1.  ]
 [ 0.    1.73 -0.58]
 [ 0.    0.    1.63]
 [ 0.    0.    0.  ]]


Q @ R =
[[ 1. -1.  1.]
 [ 1.  1.  1.]
 [ 1.  1. -1.]
 [ 1.  1.  1.]]


b =
[ 1  1 -1  0]


c =
[ 0.5  -0.87  1.22  0.71]


x =
[ 0.   -0.25  0.75]




#### $A = QR$ decomposition of Householder

In [8]:
@enforce.runtime_validation
def reflector(A: np.ndarray, i: int) -> np.ndarray:
    x = np.take(A, [i], 1)
    y =  np.concatenate([x[:i], [[np.sqrt(np.sum(np.power(x[i:],2)))]],
                        np.zeros([A.shape[0] - i - 1, 1])])
    u = (x - y)
    return np.eye(A.shape[0]) - (2 / np.squeeze(u.T @ u)) * (u @ u.T)

In [9]:
@enforce.runtime_validation
def qr_householder(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    Q = np.eye(A.shape[0])
    for i in range(A.shape[1]):
        R = reflector(A, i)
        Q = Q @ R
        A = R @ A
    return (Q, A)

In [10]:
A = np.array([[1,-1,1],
                [1,1,1],
                [1,1,-1],
                [1,1,1]])

b = np.array([1, 1, -1, 0])

[Q, R] = qr_householder(A)
c = (np.transpose(Q) @ b).flatten()
x = np.linalg.solve(R[0:R.shape[1], :], c[0:R.shape[1]])

print("A =", np.round(A, decimals=2),  '\n', sep='\n')
print("Q =", np.round(Q, decimals=2),  '\n', sep='\n')
print("R =", np.round(R, decimals=2),  '\n', sep='\n')
print("Q @ R =", np.round(Q @ R, decimals=2), '\n', sep='\n')
print("b =", np.round(b, decimals=2), '\n', sep='\n')
print("c =", np.round(c, decimals=2), '\n', sep='\n')
print("x =", np.round(x, decimals=2), '\n', sep='\n')

A =
[[ 1 -1  1]
 [ 1  1  1]
 [ 1  1 -1]
 [ 1  1  1]]


Q =
[[ 0.5  -0.87  0.   -0.  ]
 [ 0.5   0.29  0.41 -0.71]
 [ 0.5   0.29 -0.82  0.  ]
 [ 0.5   0.29  0.41  0.71]]


R =
[[ 2.    1.    1.  ]
 [ 0.    1.73 -0.58]
 [ 0.   -0.    1.63]
 [ 0.    0.    0.  ]]


Q @ R =
[[ 1. -1.  1.]
 [ 1.  1.  1.]
 [ 1.  1. -1.]
 [ 1.  1.  1.]]


b =
[ 1  1 -1  0]


c =
[ 0.5  -0.87  1.22 -0.71]


x =
[ 0.   -0.25  0.75]




## Diagonal Decomposition

#### $A = U\Sigma V^t$ decomposition

In [40]:
def inner_product(v1, v2):
    return np.sum(v1 * v2)

In [41]:
def norm_2(v):
    return np.sqrt(inner_product(v, v))

In [42]:
def normalize(v):
    return v / norm_2(v)

In [100]:
@enforce.runtime_validation
def gram_schmidt(V: np.ndarray) -> np.ndarray:
    Q = np.zeros(V.shape)
    for i in range(Q.shape[1]):
        Q[:, i] = V[:, i] 
        for j in range(i):
            Q[:, i] -= inner_product(Q[:, j], Q[:, i]) * Q[:, j]
        Q[:, i] = normalize(Q[:, i]) 
    return Q

In [113]:
def eig_vals(A):
    return np.sort(eig_vals(A.T @ A))[::-1]

In [96]:
@enforce.runtime_validation
def usv(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    U = np.zeros([A.shape[0], A.shape[0]])
    S = np.zeros(A.shape)
    V = np.shape([A.shape[1], A.shape[1]])
    
    A1 = A.T @ A
    A2 = A @ A.T
    
    lambdas = eig_vals(A1)
    
    
    
    
    
    
    
    return (U, S, V)

In [573]:
A = np.array([[1, -1, 2],
              [1, 2, -1],
              [2, 1, 1]])
np.sort(eig_vals(A.T @ A))[::-1]
#np.isclose(np.sort(eig_vals(A.T @ A))[::-1], 0)
B = A.T @ A
print(B )

[[ 6  3  3]
 [ 3  6 -3]
 [ 3 -3  6]]


In [574]:
gauss_lu(lu(B - 9 * np.eye(B.shape[0])), np.zeros(3))

array([-0.,  0.,  0.])

In [576]:
lu(B - 9 * np.eye(B.shape[0]))

(array([[ 1.,  0.,  0.],
        [-1.,  1.,  0.],
        [-1.,  0.,  1.]]), array([[-3.,  3.,  3.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]))